In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import string

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './datasets/last_seq/bfs-chr/*'

In [3]:
# file read
all_names = []
all_data = []
sequence_length = []
alpha = list(string.ascii_uppercase)
data_length = len(glob.glob(dir))
file_predix = './datasets/last_seq/bfs-chr/graph'
for index in range(data_length):
    filename = file_predix + str(index) + "-*"
    files = glob.glob(filename)
    for file in files:
        datasets = []
        all_names.append(file.split('/')[-1].replace('.txt', ''))
        for rf in open(file, 'r'):
            (u, v, w) = rf[1:-2].split(', ')
            datasets.append([alpha.index(u[1])+1, alpha.index(v[1]) +1, float(w)])
        sequence_length.append(len(datasets))
        all_data.append(datasets)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, all_names, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [6]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 200
steps_per_epoch = len(x_train)

In [7]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.kld(y_true, y_pred) # categorical_crossentropy
    return loss1 * 0.7 + loss2 * 0.3

In [8]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=custom_loss, optimizer=Adam())#decay=0.9
#lstm_autoencoder_500 = lstm_autoencoder

W1018 23:21:10.532314 140011663783680 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1018 23:21:10.567520 140011663783680 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1018 23:21:10.572081 140011663783680 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1018 23:21:11.072776 140011663783680 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1018 23:21:11.080046 14

In [9]:
def val_generator(x_val):
    idx = 0
    while True:
        yield np.array([x_val[idx]]), np.array([x_val[idx]])
        idx += 1
        if idx >= len(x_val):
            idx = 0

def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx += 1
        if idx >= len(x_train):
            idx = 0

In [10]:
hist = lstm_autoencoder.fit_generator(train_generator(x_train), epochs=200, steps_per_epoch=steps_per_epoch, verbose=1, validation_steps=len(x_val), validation_data=val_generator(x_val))

W1018 23:21:11.260695 140011663783680 deprecation.py:323] From /home/minji/anaconda3/envs/graph/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1018 23:21:12.612625 140011663783680 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/200
5733/5733 [==============================] - 52s 9ms/step - loss: 59.4739 - val_loss: 61.3151
Epoch 2/200
5733/5733 [==============================] - 48s 8ms/step - loss: 58.8361 - val_loss: 61.2788
Epoch 3/200
5733/5733 [==============================] - 48s 8ms/step - loss: 47.3874 - val_loss: 18.4891
Epoch 4/200
5733/5733 [==============================] - 48s 8ms/step - loss: 16.0368 - val_loss: 14.9025
Epoch 5/200
5733/5733 [==============================] - 48s 8ms/step - loss: 12.3924 - val_loss: 11.0430
Epoch 6/200
5733/5733 [==============================] - 49s 9ms/step - loss: 9.8033 - val_loss: 9.2461
Epoch 7/200
5733/5733 [==============================] - 49s 8ms/step - loss: 8.4951 - val_loss: 8.2454
Epoch 8/200
5733/5733 [==============================] - 48s 8ms/step - loss: 7.1298 - val_loss: 6.0721
Epoch 9/200
5733/5733 [==============================] - 49s 8ms/step - loss: 5.7658 - val_loss: 5.4181
Epoch 10/200
5733/5733 [==============================

5733/5733 [==============================] - 48s 8ms/step - loss: 0.3305 - val_loss: 0.6846
Epoch 80/200
5733/5733 [==============================] - 49s 8ms/step - loss: 0.3270 - val_loss: 0.4679
Epoch 81/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.3310 - val_loss: 0.4989
Epoch 82/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.3195 - val_loss: 0.6304
Epoch 83/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.3086 - val_loss: 0.5956
Epoch 84/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.3117 - val_loss: 0.5845
Epoch 85/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.3125 - val_loss: 0.8482
Epoch 86/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.3142 - val_loss: 0.7357
Epoch 87/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2926 - val_loss: 0.4208
Epoch 88/200
5733/5733 [==============================] - 48s 8ms/st

5733/5733 [==============================] - 48s 8ms/step - loss: 0.2116 - val_loss: 0.3795
Epoch 157/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2048 - val_loss: 0.3190
Epoch 158/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2017 - val_loss: 0.3741
Epoch 159/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2071 - val_loss: 0.3328
Epoch 160/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2012 - val_loss: 0.3491
Epoch 161/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.1926 - val_loss: 0.3185
Epoch 162/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2082 - val_loss: 0.3478
Epoch 163/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.1900 - val_loss: 0.3926
Epoch 164/200
5733/5733 [==============================] - 48s 8ms/step - loss: 0.2014 - val_loss: 0.4334
Epoch 165/200
5733/5733 [==============================] - 4

In [11]:
# MODEL SAVE
model_json = lstm_autoencoder.to_json()
filename = 'last_mse_kld_lstmae' #input('filename: ') 
with open('model_save/mse_kld_models/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model_save/mse_kld_models/weights_' +  filename + '.h5')

In [12]:
import json
with open('model_save/mse_kld_models/last_kld_history.json', 'w') as f:
    json.dump(hist.history, f)

In [13]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

loss_ax.plot(hist.history['val_loss'], 'b', label='val loss')
loss_ax.plot(hist.history['loss'], 'r', label='train loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

plt.title('kld mse')
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
loaded_model = lstm_autoencoder

In [15]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

0.5202376774581481


In [5]:
filename = "latest_mse_kld_lstmae1"
# MODEL LOAD
loaded_model = model_from_json(open('model_save/mse_kld_models/' +filename + '.json').read())
loaded_model.load_weights('model_save/mse_kld_models/weights_' + filename + '.h5')

W1017 20:05:39.672357 139985310242560 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1017 20:05:39.683409 139985310242560 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1017 20:05:39.685610 139985310242560 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1017 20:05:40.291379 139985310242560 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [ ]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [23]:
model_path = 'model_save/mse_kld_models/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min', period=100)